<a href="https://colab.research.google.com/github/oj-o/Art-Tech-Science/blob/main/Sound-OSC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: 음악에서 악기를 인식하여 osc가 가능하도록

!pip install python-osc librosa
!apt-get update
!apt-get install -y ffmpeg

import librosa
import numpy as np
from pythonosc import udp_client
import time

# --- Configuration ---
OSC_IP = "127.0.0.1"  # Replace with the IP address of your OSC server
OSC_PORT = 5005       # Replace with the port of your OSC server
AUDIO_FILE = "your_music_file.wav" # Replace with your audio file

# --- OSC Setup ---
try:
    client = udp_client.SimpleUDPClient(OSC_IP, OSC_PORT)
    print(f"OSC client connected to {OSC_IP}:{OSC_PORT}")
except Exception as e:
    print(f"Error connecting to OSC server: {e}")
    client = None

# --- Instrument Recognition Function (Placeholder) ---
# This is where the core logic for instrument recognition goes.
# This is a highly complex task and would typically involve:
# 1. Feature extraction (e.g., MFCCs, spectral features)
# 2. Training a machine learning model (e.g., using datasets like IRMAS, OpenMIC)
# 3. Predicting the instrument(s) present in a given audio segment.
# This placeholder function just returns a dummy instrument for demonstration.
def recognize_instrument(audio_segment, sr):
    """
    Placeholder function for instrument recognition.
    In a real application, this would use a trained model.
    """
    # In a real scenario, you would analyze the audio_segment
    # using trained models to determine the instrument(s).
    # For this example, we'll just return a dummy value.
    # You could potentially analyze dominant frequencies, spectral centroids, etc.
    # to make a very basic guess without a full ML model, but accuracy would be low.

    # Dummy logic: If the audio segment has a certain characteristic (e.g., loud),
    # pretend it's a piano. This is NOT a real instrument recognizer.
    if np.mean(np.abs(audio_segment)) > 0.01:
        return "piano"
    else:
        return "other"


# --- Audio Processing and OSC Sending ---
def process_audio_and_send_osc(audio_path):
    if client is None:
        print("OSC client not initialized. Cannot send messages.")
        return

    try:
        y, sr = librosa.load(audio_path, sr=None) # Load audio file

        # Define chunk size for processing (e.g., 1 second)
        chunk_size = sr  # Process in 1-second chunks

        print(f"Processing audio file: {audio_path}")
        print(f"Sample rate: {sr}")
        print(f"Duration: {librosa.get_duration(y=y, sr=sr):.2f} seconds")

        for i in range(0, len(y), chunk_size):
            audio_chunk = y[i:i + chunk_size]
            current_time = i / sr # Time in seconds from the start of the audio

            # Recognize instrument in the current chunk
            instrument = recognize_instrument(audio_chunk, sr)

            # Send OSC message based on recognized instrument
            if instrument == "piano":
                # Example OSC message: Send a message to the address "/instrument/piano"
                # You could add more parameters like intensity, pitch, etc.
                print(f"Time: {current_time:.2f}s - Detected: Piano. Sending OSC message /instrument/piano")
                client.send_message("/instrument/piano", 1) # Send a value of 1 (e.g., indicating presence)
            elif instrument == "other":
                 # Example: Send a message for other instruments or no dominant instrument
                 # print(f"Time: {current_time:.2f}s - Detected: Other. Sending OSC message /instrument/other")
                 # client.send_message("/instrument/other", 0) # Send 0

            # Add a small delay to simulate real-time processing (optional)
            time.sleep(chunk_size / sr)

    except FileNotFoundError:
        print(f"Error: Audio file not found at {audio_path}")
    except Exception as e:
        print(f"An error occurred during audio processing: {e}")

# --- Main Execution ---
if __name__ == "__main__":
    # Create a dummy audio file for demonstration if it doesn't exist
    # In a real scenario, you would have your actual music file.
    import soundfile as sf
    if not os.path.exists(AUDIO_FILE):
        print(f"Creating a dummy audio file: {AUDIO_FILE}")
        # Generate a simple sine wave for demonstration
        sr = 22050
        duration = 5
        frequency = 440
        t = np.linspace(0., duration, int(sr * duration))
        amplitude = 0.5
        dummy_audio = amplitude * np.sin(2. * np.pi * frequency * t)
        sf.write(AUDIO_FILE, dummy_audio, sr)


    import os
    if os.path.exists(AUDIO_FILE):
        process_audio_and_send_osc(AUDIO_FILE)
    else:
        print("Could not create dummy audio file. Please provide a valid audio file path.")

